In [15]:
%load_ext autoreload
%autoreload 2
#%matplotlib tk

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
import unstructured_airfoil_mesh_inverter as meshinv
import Basis
import NURBS_patch
import matplotlib.pyplot as plt
import numpy as np
import sys
import math
from tqdm import tnrange, tqdm_notebook

In [17]:
CONST_INPUT_MSH_FILE = "NACA0012_unstructured_mesh/NACA0012.msh"
CONST_NURBS_PATCH_FILE = "/Users/jm-034232/Documents/McGill/Research/DPGSolver/input/input_files/euler/steady/NURBS_Airfoil/geometry_parameters_NACA0012_tests_reference.geo"

In [18]:
node_list, elements_dict = meshinv.read_mesh_file(CONST_INPUT_MSH_FILE)

patch_information = meshinv.read_patch_file(CONST_NURBS_PATCH_FILE)

R_ij = Basis.get_NURBS_basis_functions(patch_information["Control_Points_and_Weights"], 
    patch_information["P"], patch_information["Q"], patch_information["xiVector"], 
    patch_information["etaVector"])

R_ij_grad = Basis.get_grad_NURBS_basis_functions(patch_information["Control_Points_and_Weights"], 
    patch_information["P"], patch_information["Q"], patch_information["xiVector"], 
    patch_information["etaVector"])

numI = len(patch_information["Control_Points_and_Weights"])
numJ = len(patch_information["Control_Points_and_Weights"][0])

R_ij_grad_xi, R_ij_grad_eta = [], []

for i in range(numI):
    row1, row2 = [], []

    for j in range(numJ):
        row1.append(R_ij_grad[i][j][0])
        row2.append(R_ij_grad[i][j][1])

    R_ij_grad_xi.append(row1)
    R_ij_grad_eta.append(row2)


nurbs_patch = lambda xi, eta: NURBS_patch.NURBS_patch(xi, eta, 
    patch_information["Control_Points_and_Weights"], R_ij)

nurbs_patch_grad_xi = lambda xi, eta: NURBS_patch.NURBS_patch(xi, eta, 
    patch_information["Control_Points_and_Weights"], R_ij_grad_xi)

nurbs_patch_grad_eta = lambda xi, eta: NURBS_patch.NURBS_patch(xi, eta, 
    patch_information["Control_Points_and_Weights"], R_ij_grad_eta)


guess_pt_grid = meshinv.initialize_guess_point_grid(nurbs_patch)

In [20]:

# Invert the node elements.
# - Try inverting all the points together

for i in tnrange(len(node_list)):
    
    pt_physical = node_list[i].pt_physical
    xi_point = meshinv.invert_physical_point(guess_pt_grid, pt_physical, nurbs_patch, nurbs_patch_grad_xi, nurbs_patch_grad_eta)


KeyboardInterrupt: 